In [1]:
#11章標準ライブラリめぐりーPartII
#11.1出力整形
import reprlib

In [2]:
reprlib.repr(set('supercalifragilisticexpilalidocious'))

"{'a', 'c', 'd', 'e', 'f', 'g', ...}"

In [3]:
import pprint

In [4]:
t=[[[['black','cyan'],'white',['green','red']],[['magenta','yellow'],'blue']]]

In [6]:
pprint.pprint(t,width=30)

[[[['black', 'cyan'],
   'white',
   ['green', 'red']],
  [['magenta', 'yellow'],
   'blue']]]


In [7]:
import textwrap

In [8]:
doc="""The wrap() method is just like fill()except that it returns
alist of strings instead of one big string with newlines to separate
the warapped lines."""

In [9]:
print(textwrap.fill(doc,width=40))#(wrap()はfille()そっくりのメソッドだが、折り返した行は
改行区切りの単一の大きな文字列で返さず、文字列のリストで返す)

The wrap() method is just like
fill()except that it returns alist of
strings instead of one big string with
newlines to separate the warapped lines.


In [10]:
import locale

In [11]:
locale.setlocale(locale.LC_ALL,'English_United States.1252')

'English_United States.1252'

In [15]:
conv=locale.localeconv()
x=1234567.8

In [13]:
locale.format("%d",x,grouping=True)

'1,234,567'

In [16]:
locale.format_string("%s%.*f",(conv['currency_symbol'],
                               conv['frac_digits'],x),grouping=True)

'$1,234,567.80'

In [17]:
#11.2テンプレート
from string import Template

In [18]:
t=Template('${village}folk send $$10 to $cause.')

In [19]:
t.substitute(village='Nottingham',cause='the ditch fund')
#ノッティンガムの人々はドブファンドに$10送る

'Nottinghamfolk send $10 to the ditch fund.'

In [20]:
t=Template('Return the $item to $owner.')

In [21]:
d=dict(item='unladen swallow')

In [22]:
t.substitute(d)

KeyError: 'owner'

In [23]:
t.safe_substitute(d)
#(空荷のツバメは$ownerに戻してください)

'Return the unladen swallow to $owner.'

In [35]:
import time, os.path

In [36]:
photofiles=['img_1074.jpg','img_1076.jpg','img_1077.jpg']

In [37]:
class BatchRename(Template):
    delimiter='%'

In [38]:
fmt=input('どのようにリネームしますか(%d-日付 %n-番号 %f-形式):')

どのようにリネームしますか(%d-日付 %n-番号 %f-形式):Ashley_%n%f


In [39]:
t=BatchRename(fmt)

In [40]:
date=time.strftime('%d%b%y')

In [43]:
for i , filename in enumerate(photofiles):
    base,ext=os.path.splitext(filename)
    newname=t.substitute(d=date,n=i,f=ext)
    print('{0}-->{1}'.format(filename,newname))

img_1074.jpg-->Ashley_0.jpg
img_1076.jpg-->Ashley_1.jpg
img_1077.jpg-->Ashley_2.jpg


In [44]:
#11.3バイナリデータレコードの処理
import struct

In [47]:
with open('myfile.zip','rb') as f:
    data=f.read()

In [51]:
start=0
for i in range(3): #先頭から三つのファイルのヘッダを示す
    start +=14
    fields=struct.unpack('<IIIHH',data[start:start+16])
    crc32,comp_size,uncomp_size,filenamesize,extra_size=fields
    
    start +=16
    filename=data[start:start+filenamesize]
    start +=filenamesize
    extra=data[start:start+extra_size]
    print(filename,hex(crc32),comp_size,uncomp_size)
    
    start +=extra_size+comp_size #次のヘッダまでスキップ

b'myfile.txt' 0x8c9cc4ff 28 36
b'' 0xc4ff4c6f 1871004 2359296


error: unpack requires a buffer of 16 bytes

In [52]:
#11.4マルチスレッド
import threading,zipfile

In [66]:
class AsyncZip(threading.Thread):
    def __init__(self,infile,outfile):
        threading.Thread.__init__(self)
        self.infile=infile
        self.outfile=outfile
    def run(self):
        f=zipfile.ZipFile(self.outfile,'w',zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of:',self.infile)

In [67]:
background=AsyncZip('mydata.txt','myarchive.zip')

In [68]:
background.start()

Finished background zip of: mydata.txt


In [69]:
print('メインプログラムは表で動き続けています。')

メインプログラムは表で動き続けています。


In [70]:
background.join()#バックグランドタスクの終了を持つ

In [71]:
print('メインプログラムはバックグランド処理の終了まで待ってました。')

メインプログラムはバックグランド処理の終了まで待ってました。


In [72]:
#11.5ログ取り
import logging
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found','server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

ERROR:root:Error occurred
CRITICAL:root:Critical error -- shutting down


In [73]:
#11.6弱参照
import weakref,gc

In [87]:
class A:
    def __init__(self,value):
        self.value=value
    def __repr__(self):
        return str(self.value)

In [88]:
a=A(10) #参照を生成する

In [89]:
d=weakref.WeakValueDictionary()

In [90]:
d['primary']=a #参照を生成しない

In [91]:
d['primary'] #オブジェクトが生きていれば取ってくる

10

In [95]:
del a #参照を削除

NameError: name 'a' is not defined

In [96]:
gc.collect() #ガベージコレクションを実行

65

In [98]:
del d['primary'] #エントリは自動的に削除されている

In [99]:
d['primary']

KeyError: 'primary'

In [100]:
#11.7 リスト操作のツール
from array import array

In [101]:
a=array('H',[4000,10,700,22222])

In [102]:
sum(a)

26932

In [103]:
a[1:3]

array('H', [10, 700])

In [104]:
from collections import deque

In [105]:
d=deque(["task1","task2","task3"])

In [106]:
d.append("task4")

In [107]:
print("Handling",d.popleft())

Handling task1


In [108]:
unsearched=deque([starting_node])

NameError: name 'starting_node' is not defined

In [109]:
def breadth_first_search(unsearched):
    node=unsearched.popleft()
    for m in gen_moves(node):
        if is_goal(m):
            return m
        unsearched.append(m)

In [110]:
import bisect

In [111]:
scores=[(100,'perl'),(200,'tcl'),(400,'lua'),(500,'python')]

In [112]:
scores

[(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]

In [113]:
from heapq import heapify,heappop,heappush

In [114]:
data=[1,3,5,7,9,2,4,6,8,0]

In [115]:
heapify(data) #リストをヒープ順に並べ替える

In [116]:
heappush(data,-5) #エントリを追加

In [117]:
[heappop(data) for i in range(3)] #最小のエントリから三つ取得

[-5, 0, 1]

In [118]:
#11.8 10進数の浮動小数点計算
from decimal import *

In [119]:
round(Decimal('0.70') * Decimal('1.05'),2)

Decimal('0.74')

In [120]:
round(.70 * 1.05,2)

0.73

In [121]:
Decimal('1.00') % Decimal('.10')

Decimal('0.00')

In [122]:
1.00 % 0.10

0.09999999999999995

In [123]:
sum([Decimal('0.1')]*10) == Decimal('1.0')

True

In [124]:
sum([0.1]*10)==1.0

False